<a href="https://colab.research.google.com/github/ailab-nda/NLP/blob/main/pytorch_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning による物体追跡

本日の授業は課題付与ですが、内容はこのノートを実行することとなります。

## 1. 準備

###ライブラリのインストール

In [ ]:
pip install scikit-learn==0.22.2 filterpy==1.1.0

### プログラムのダウンロード

In [ ]:
!git clone https://github.com/cfotache/pytorch_objectdetecttrack.git
% cd pytorch_objectdetecttrack/config/
! bash download_weights.sh

### ライブラリのインポート

In [ ]:
% cd /content/pytorch_objectdetecttrack

In [ ]:
from models import *
from utils import *

import os, sys, time, datetime, random
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.autograd import Variable

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

### モデル(Yolo) のセットアップ

In [ ]:
config_path='config/yolov3.cfg'
weights_path='config/yolov3.weights'
class_path='config/coco.names'
img_size=416
conf_thres=0.8
nms_thres=0.4

# Load model and weights
model = Darknet(config_path, img_size=img_size)
model.load_weights(weights_path)
model.cuda()
model.eval()
classes = utils.load_classes(class_path)
Tensor = torch.cuda.FloatTensor

### 物体検出用関数の定義

In [ ]:
def detect_image(img):
    # scale and pad image
    ratio = min(img_size/img.size[0], img_size/img.size[1])
    imw = round(img.size[0] * ratio)
    imh = round(img.size[1] * ratio)
    img_transforms = transforms.Compose([ transforms.Resize((imh, imw)),
         transforms.Pad((max(int((imh-imw)/2),0), max(int((imw-imh)/2),0), max(int((imh-imw)/2),0), max(int((imw-imh)/2),0)),
                        (128,128,128)),
         transforms.ToTensor(),
         ])
    # convert image to Tensor
    image_tensor = img_transforms(img).float()
    image_tensor = image_tensor.unsqueeze_(0)
    input_img = Variable(image_tensor.type(Tensor))
    # run inference on the model and get detections
    with torch.no_grad():
        detections = model(input_img)
        detections = utils.non_max_suppression(detections, 80, conf_thres, nms_thres)
    return detections[0]

## 2. 物体検出

pytorch_objectdetectrack の中の中の images にサンプルイメージが入っています。Intersection-Counts.jpg 以外でも試してみてください。

In [ ]:
# load image and get detections
img_path = "images/Intersection-Counts.jpg"
prev_time = time.time()
img = Image.open(img_path)
detections = detect_image(img)
inference_time = datetime.timedelta(seconds=time.time() - prev_time)
print ('Inference Time: %s' % (inference_time))

# Get bounding-box colors
cmap = plt.get_cmap('tab20b')
colors = [cmap(i) for i in np.linspace(0, 1, 20)]

img = np.array(img)
plt.figure()
fig, ax = plt.subplots(1, figsize=(12,9))
ax.imshow(img)

pad_x = max(img.shape[0] - img.shape[1], 0) * (img_size / max(img.shape))
pad_y = max(img.shape[1] - img.shape[0], 0) * (img_size / max(img.shape))
unpad_h = img_size - pad_y
unpad_w = img_size - pad_x

if detections is not None:
    unique_labels = detections[:, -1].cpu().unique()
    n_cls_preds = len(unique_labels)
    bbox_colors = random.sample(colors, n_cls_preds)
    # browse detections and draw bounding boxes
    for x1, y1, x2, y2, conf, cls_conf, cls_pred in detections:
        box_h = ((y2 - y1) / unpad_h) * img.shape[0]
        box_w = ((x2 - x1) / unpad_w) * img.shape[1]
        y1 = ((y1 - pad_y // 2) / unpad_h) * img.shape[0]
        x1 = ((x1 - pad_x // 2) / unpad_w) * img.shape[1]
        color = bbox_colors[int(np.where(unique_labels == int(cls_pred))[0])]
        bbox = patches.Rectangle((x1, y1), box_w, box_h, linewidth=2, edgecolor=color, facecolor='none')
        ax.add_patch(bbox)
        plt.text(x1, y1, s=classes[int(cls_pred)], color='white', verticalalignment='top',
                bbox={'color': color, 'pad': 0})
plt.axis('off')
# save image
plt.savefig(img_path.replace(".jpg", "-det.jpg"), bbox_inches='tight', pad_inches=0.0)
plt.show()

## 3. 物体追跡

動画の長さは１分程度がお勧めです（それ以上長いと授業時間内では終わりません）。

### 動画の取得

※ 自分で動画が用意できる人は、左のパネルを「ファイル」にした上で video.mp4 という名前にした動画ファイルをドラッグして置いてください。

以下では、Youtube から数分の動画をダウンロードして必要な部分（20秒程度）を切り出します。

*   下記の url 変数に設置したい動画の URL を記述します。
*   下記の start_time に切り出したい開始時間を記入します。
*   下記の duration に何秒間分切り出すかを記入します。

In [ ]:
!rm /content/video_org.mp4
!pip install youtube-dl
import youtube_dl

# 例１：野球
url = 'https://www.youtube.com/watch?v=VCHl3Yrnfjs' 
start_time = "00:00:06"
duration = "00:00:20"

# 例２：バイク
#url = 'https://www.youtube.com/watch?v=wnRH3-CIk4I' 
#start_time = "00:00:10"
#duration = "00:00:20"

vdl_opts = {'outtmpl':'/content/video_org.mp4', 'format':'bestvideo'}
vdl = youtube_dl.YoutubeDL(vdl_opts)
vdl.extract_info(url)

!yes|ffmpeg -ss $start_time -i "/content/video_org.mp4" -t $duration -c copy "/content/video.mp4"

[download]  23.6% of 50.60MiB at 80.92KiB/s ETA 08:09

### 物体追跡
ここでやっていること：動画を１フレームずつに分けて物体検出をし、結果を統合して動画を作り直します。

In [ ]:
img_path = '/content/img'
if not os.path.exists(img_path):
  os.mkdir(img_path)
!rm /content/img/*

videopath = '/content/video.mp4'

import cv2
from IPython.display import clear_output

cmap = plt.get_cmap('tab20b')
colors = [cmap(i)[:3] for i in np.linspace(0, 1, 20)]

# initialize Sort object and video capture
from sort import *
vid = cv2.VideoCapture(videopath)
mot_tracker = Sort() 
ii = 0
while(True):
    ret, frame = vid.read()
    if ret == False:
        break
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    pilimg = Image.fromarray(frame)
    detections = detect_image(pilimg)

    img = np.array(pilimg)
    pad_x = max(img.shape[0] - img.shape[1], 0) * (img_size / max(img.shape))
    pad_y = max(img.shape[1] - img.shape[0], 0) * (img_size / max(img.shape))
    unpad_h = img_size - pad_y
    unpad_w = img_size - pad_x
    if detections is not None:
        tracked_objects = mot_tracker.update(detections.cpu())

        unique_labels = detections[:, -1].cpu().unique()
        n_cls_preds = len(unique_labels)
        for x1, y1, x2, y2, obj_id, cls_pred in tracked_objects:
            box_h = int(((y2 - y1) / unpad_h) * img.shape[0])
            box_w = int(((x2 - x1) / unpad_w) * img.shape[1])
            y1 = int(((y1 - pad_y // 2) / unpad_h) * img.shape[0])
            x1 = int(((x1 - pad_x // 2) / unpad_w) * img.shape[1])

            color = colors[int(obj_id) % len(colors)]
            color = [i * 255 for i in color]
            cls = classes[int(cls_pred)]
            cv2.rectangle(frame, (x1, y1), (x1+box_w, y1+box_h), color, 4)
            cv2.rectangle(frame, (x1, y1-35), (x1+len(cls)*19+60, y1), color, -1)
            cv2.putText(frame, cls + "-" + str(int(obj_id)), (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 3)
    ii = ii + 1
    #print("ii=", ii)
    frame = cv2.cvtColor(frame.astype(np.float32), cv2.COLOR_RGB2BGR)
    cv2.imwrite("/content/img/" + str(ii).zfill(5) + ".jpg", frame)

### 全てのフレームを結合して動画にする

In [ ]:
!yes|ffmpeg -r 30 -i '/content/img/%05d.jpg' -vcodec libx264 -pix_fmt yuv420p -r 30 '/content/tracking.mp4'

## トラッキング結果（動画）の表示

この画面で再生できるようにしましたが、防大だと回線が細くてカクカクです。うまく見られない場合は、画面左端のフォルダのマークをクリックすると今回使用したファイルの一覧が出ます。


*   video_org.mp4: ダウンロードしたオリジナルの動画
*   video.mp4: 必要な秒数だけ抜き出した動画
*   tracking.mp4: トラッキング結果の動画

各動画をダウンロードして手元で見てください。



In [ ]:
from IPython.display import HTML
from base64 import b64encode

video_path = "/content/tracking.mp4"
 
def show_video(video_path, video_width = 600):
   
  video_file = open(video_path, "r+b").read()
 
  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  return HTML(f"""<video width={video_width} controls><source src="{video_url}"></video>""")
 
show_video(video_path)